# Denue y hospitales

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import folium
import vaex as vx
import csv

In [26]:
with open("INEGI/Denue_covid/actividades_no_esenciales_01_14.csv",encoding='UTF-16') as f:
    df = pd.read_csv(f)
temp_no_esen = df[(df.cve_ent == 9)]
denue_no_esen = temp_no_esen[['nom_estab', 'codigo_act', 'nombre_act', 'cve_ent','cve_mun', 'cve_loc','ageb','latitud','longitud', 'es_esencial']]
denue_no_esen['nom_estab'] = denue_no_esen['nom_estab'].astype(str)
denue_no_esen['nombre_act'] = denue_no_esen['nombre_act'].astype(str)
denue_no_esen['ageb'] = denue_no_esen['ageb'].astype(str)

gdf_no_esen = gpd.GeoDataFrame(denue_no_esen, geometry=gpd.points_from_xy(denue_no_esen.longitud, denue_no_esen.latitud))
gdf_no_esen.crs = {'init' :'epsg:4326'}
gdf_no_esen.to_file("denue_no_esen.json", driver="GeoJSON")


/home/edman/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/edman/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/edman/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/edman/.loca

In [27]:
with open("INEGI/Denue_covid/actividades_esenciales_01_14.csv",encoding='UTF-16') as f:
    df = pd.read_csv(f, error_bad_lines=False)
temp_esen = df[(df.cve_ent == 9)]
denue_esen = temp_esen[['nom_estab', 'codigo_act', 'nombre_act', 'cve_ent','cve_mun', 'cve_loc','ageb','latitud','longitud', 'es_esencial']]

denue_esen['nom_estab'] = denue_esen['nom_estab'].astype(str)
denue_esen['nombre_act'] = denue_esen['nombre_act'].astype(str)
denue_esen['ageb'] = denue_esen['ageb'].astype(str)

gdf_esen = gpd.GeoDataFrame(denue_esen, geometry=gpd.points_from_xy(denue_esen.longitud, denue_esen.latitud))
gdf_esen.crs = {'init' :'epsg:4326'}
gdf_esen.to_file("denue_esen.json", driver="GeoJSON")

b'Skipping line 717256: expected 42 fields, saw 45\n'
b'Skipping line 857336: expected 42 fields, saw 44\nSkipping line 857337: expected 42 fields, saw 44\n'
/home/edman/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (14,26,28,30,33,34,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/edman/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/edman/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [28]:
esen = gpd.read_file('denue_esen.json')
no_esen = gpd.read_file('denue_no_esen.json')

denue = esen.append(no_esen, ignore_index = True) 

denue['cve_ent'] = denue['cve_ent'].astype(np.int64)
denue['cve_mun'] = denue['cve_mun'].astype(np.int64)
denue['cve_loc'] = denue['cve_loc'].astype(np.int64)
denue['ageb'] = denue['ageb'].astype(str)
denue['longitud'] = denue['longitud'].astype(np.float64)
denue.crs = {'init' :'epsg:4326'}
denue.to_file("denue.json", driver="GeoJSON")


ERROR:MainThread:fiona._env:denue.json: No such file or directory


In [2]:
zonas_cdmx = gpd.read_file('zonas_cdmx.json')
denue = gpd.read_file('denue.json')


In [3]:
denue_coord = gpd.GeoDataFrame(denue, geometry=gpd.points_from_xy(denue.longitud, denue.latitud))
denue_coord.crs = {'init' :'epsg:4326'}
denue_coord_no_esen = denue_coord[(denue_coord.es_esencial == 0)]
denue_coord_esen = denue_coord[(denue_coord.es_esencial == 1)]


/home/edman/.local/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
#COnteo denue no esen

sjoined_denue_no_esen = gpd.sjoin(denue_coord_no_esen, zonas_cdmx, op='within')

grouped_denue_no_esen = sjoined_denue_no_esen.groupby('CVE_AGEB').size()
df_denue_no_esen = grouped_denue_no_esen.to_frame().reset_index()
df_denue_no_esen.columns = ['CVE_AGEB', 'no_esen_count']

merged_no_esen = zonas_cdmx.merge(df_denue_no_esen, on='CVE_AGEB', how='outer')

merged_no_esen.to_file("denue_no_esen.json", driver="GeoJSON")

/home/edman/.local/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [5]:
#COnteo denue esen

sjoined_denue_esen = gpd.sjoin(denue_coord_esen, zonas_cdmx, op='within')

grouped_denue_esen = sjoined_denue_esen.groupby('CVE_AGEB').size()
df_denue_esen = grouped_denue_esen.to_frame().reset_index()
df_denue_esen.columns = ['CVE_AGEB', 'esen_count']

merged_esen = zonas_cdmx.merge(df_denue_esen, on='CVE_AGEB', how='outer')

merged_esen.to_file("denue_esen.json", driver="GeoJSON")

/home/edman/.local/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [29]:
denue_restaurantes = denue[(denue.codigo_act == 722511) | (denue.codigo_act == 722512) | (denue.codigo_act == 722513) |(denue.codigo_act == 722514) |(denue.codigo_act == 722516) |(denue.codigo_act == 722517) |(denue.codigo_act == 722518)]
denue_restaurantes.crs = {'init' :'epsg:4326'}
denue_restaurantes.to_file("denue_restaurantes.json", driver="GeoJSON")

In [7]:
gdf_hosp = gpd.read_file('Hospitales/hospitales-y-centros-de-salud.geojson')
gdf_hosp.crs = {'init' :'epsg:4326'}
gdf_hosp.to_file("hospitales.json", driver="GeoJSON")